# NOAA
- request example: https://www.ncei.noaa.gov/access/services/data/v1?dataset=local-climatological-data&dataTypes=HourlyWindSpeed&dataTypes=HourlyDryBulbTemperature&stations=72530094846&startDate=2018-05-01&endDate=2018-05-31
- list of stationsids: https://www.itl.nist.gov/div898/winds/asos-wx/WBAN-MSC.TXT
 - list of WBAN and lat/long: https://www.epa.gov/sites/default/files/documents/STATION_LOCATIONS.PDF
- Other Notes: https://www.ncei.noaa.gov/access/services/support/v3/datasets.json

In [7]:
#needed to make web requests
import requests

#store the data we get as a dataframe
import pandas as pd

#convert the response as a strcuctured json
import json

#mathematical operations on lists
import numpy as np

#parse the datetimes we get from NOAA
from datetime import datetime

import urllib

import os

pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)

## Setup

### Read in Station Names

In [103]:
cwd = os.getcwd()

download_fldr = os.path.join(cwd, 'downloads')

# sites
stations_link = 'https://www.itl.nist.gov/div898/winds/asos-wx/WBAN-MSC.TXT'

# read in html byte
i = requests.get(stations_link)
content = i.content
decoded_content  = str(content,'UTF-8')

end_list = []

#skip first 6 rows since it doesn't contain actual data
#read cols in first then create list with all data
#Couldnt pd.read_csv bc some of the names are 3+ words and it would skip the >2+ NAMEs
for n,i in enumerate(decoded_content.splitlines()[6:]):
    if n ==0:
        cols = list(i.split())
    else:
        nums = i.split()[0:2]
        names = " ".join(i for i in i.split()[2:])
        
        nums.append(names)

        end_list.append(nums)

df = pd.DataFrame(end_list, columns=cols)
df = df[df['NAME'].notna()]
df['AWSMSC_WBAN'] = df['AWSMSC'] + df['WBAN']

### Houston Stations

In [107]:
hou_stations = df[df.NAME.str.contains('HOUSTON')].reset_index(drop=True)
hou_stations

,AWSMSC,WBAN,NAME,AWSMSC_WBAN
0,722429,53910,HOUSTON HOOKS MEMORIAL AP,72242953910
1,722430,12960,HOUSTON INTERCONTINENTAL AP,72243012960
2,722433,12969,HOUSTON LAKESIDE ARP,72243312969
3,722435,12918,HOUSTON WILLIAM P HOBBY AP,72243512918
4,722436,12906,HOUSTON ELLINGTON AFB,72243612906


## Data Download

In [5]:
awsmsc_wban = iah_id
strt_dte = '2020-01-01'
end_dte = '2020-12-01'

In [21]:
noaa_api_call = f'https://www.ncei.noaa.gov/access/services/data/v1?dataset=local-climatological-data&dataTypes=HourlyWindSpeed&dataTypes=HourlyDryBulbTemperature&stations={awsmsc_wban}&startDate={strt_dte}&endDate={end_dte}'
noaa_api_call

'https://www.ncei.noaa.gov/access/services/data/v1?dataset=local-climatological-data&dataTypes=HourlyWindSpeed&dataTypes=HourlyDryBulbTemperature&stations=72243012960&startDate=2020-01-01&endDate=2020-12-01'

In [ ]:
station = ''
data_type = 'local-climatological-data'

In [25]:
try:
   os.makedirs("downloads")
except FileExistsError:
   # directory already exists
   pass

with open(os.path.join(download_fldr, 'test_data' + '.csv'), 'wb') as file:
    response = requests.get(noaa_api_call, allow_redirects=True)
    file.write(response.content)